In [ ]:
import torch  
import os  
import numpy as np  
import pandas as pd  
from tqdm import tqdm  
import seaborn as sns 
from pylab import rcParams  
import matplotlib.pyplot as plt  
from matplotlib import rc  
from sklearn.preprocessing import MinMaxScaler  
from pandas.plotting import register_matplotlib_converters  
from torch import nn, optim 
import matplotlib
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)


In [ ]:
df= pd.read_excel('E:\covid-data.xlsx', usecols='A,D',index_col=0,parse_dates=True)
df.dropna(inplace=True)
print(len(df))
df.head()

In [15]:
y = df['Active Cases'].values.astype(float)

test_size = 100
window_size = 14

train_data = y[:-test_size]
test_data = y[-test_size:]

In [ ]:
print(f'Train: {len(train_data)}')
print(f'Test:  {len(test_data)}')

In [ ]:
plt.figure(figsize=(12,4))
plt.ylabel('number of infected people')
plt.grid(True)
plt.autoscale(axis='x',tight=True)
plt.plot(df['Active Cases'])
plt.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))

train_norm = scaler.fit_transform(train_data.reshape(-1, 1))
test_norm = scaler.fit_transform(test_data.reshape(-1, 1))
print(f'First item, original: {train_data[0]}')
print(f'First item, scaled:  {train_norm[0]}')


In [19]:

def create_dataset(dataset, timestep ):
    dataX, dataY = [], []
    for i in range(len(dataset)-timestep -1):
        a = dataset[i:(i+timestep )]
        dataX.append(a)
        dataY.append(dataset[i + timestep])
    return np.array(dataX),np.array(dataY)

trainX1,trainY1  = create_dataset(train_norm,window_size)
testX1,testY1  = create_dataset(test_norm,window_size)

trainX = np.reshape(trainX1, (trainX1.shape[0], trainX1.shape[1], 1))
testX = np.reshape(testX1, (testX1.shape[0], testX1.shape[1], 1))


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
import tensorflow as tf
from sklearn import metrics
#def mape(y_true, y_pred):
#    return K.mean(K.abs((y_true - y_pred) / y_true), axis=-1)

units =30
rate=0.3
epochs=50
batch_size=64
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
model = Sequential()
model.add(LSTM(units = units,activation='tanh', input_shape = (None,1)))
model.add(Dropout(rate=rate))
model.add(Dense(units = 1,activation='linear'))
model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mape'])



In [ ]:
history = model.fit(trainX, trainY1, epochs=epochs, batch_size=batch_size, verbose=1,
                        validation_data=(testX, testY1), )  

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='test_loss')
plt.legend()
plt.show()


In [ ]:
train_pre=model.predict(trainX)
plt.figure(figsize=(15, 8))
train_pre=scaler.inverse_transform(train_pre)
trainY1_pre=scaler.inverse_transform(trainY1)
plt.plot(range(1363),trainY1_pre,range(1363),train_pre)
plt.legend(['true','pre'])
plt.show()


In [ ]:
test_pre=model.predict(testX)
plt.figure(figsize=(15, 8))
test_pre=scaler.inverse_transform(test_pre)
testY1_true=scaler.inverse_transform(testY1)
plt.plot(range(85),testY1_true,range(85),test_pre)
plt.legend(['true','pre'])
plt.show()
#print(test_pre)

In [ ]:

plt.plot(
    df.index[:len(train_data)],
    train_data.flatten(),
    label='Historical Daily Cases')
 
plt.plot(
    df.index[len(train_data):len(train_data) + len(testY1_true)],
    testY1_true,
    label='Real Daily Cases')
 
plt.plot(
    df.index[len(train_data):len(train_data) + len(test_pre)],
    test_pre,
    label='Predicted Daily Cases')
 
plt.legend()
plt.show()


In [ ]:
n=window_size
testpredict=model.predict(testX)[-n-1:-1]

a=[]
for i in range(len(testpredict)):
    a.append(testpredict[i][0])
print(a)


#预测
def Perdict(a,n):
    Input=a[-n:]
    Input = np.array(Input)
    Input = Input.astype('float32')
    Input = Input.reshape(-1, 1)

    Input = np.reshape(Input[-n:], (Input.shape[0], Input.shape[1], 1))

    featruePredict = model.predict(Input)

    featruePredict = scaler.inverse_transform(featruePredict)

    return featruePredict[0][0]


for i in range(len(testpredict)):
    t=Perdict(a,n)
    a.append(t)
    
a = a[14:27]
print("预测结果为：",a)
plt.plot(a)
plt.ticklabel_format(style='plain')
plt.show()